In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
# Function to load data from folders
def load_data_from_folders(main_folder_path):
    data = []
    for category_folder in os.listdir(main_folder_path):
        category_label = category_folder  # Assuming folder name is the category label
        category_path = os.path.join(main_folder_path, category_folder)
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                data.append({'text': content, 'category': category_label})
    return pd.DataFrame(data)

# Main folder containing subfolders for each category
main_folder_path = '/kaggle/input/sanad-dataset'

# Load data from folders
df = load_data_from_folders(main_folder_path)

In [7]:
df

,text,category
0,"دبي - ""الخليج"":حصدت شعاع كابيتال جائزة ""أفضل ش...",Finance
1,أبوظبي - علي أسعد: تراجعت أسواق المال في الدول...,Finance
2,استأنفت أسواق الأسهم المحلية أمس تحركها باتجاه...,Finance
3,دبي «الخليج»: أعلنت شركة تكافل الإمارات عن بدء...,Finance
4,تحتفل شركة طاقة الخليج البحرية، التي تتخذ من د...,Finance
...,...,...
45495,على هامش مشاركتها في أسبوع جيتكس للتقنية أعلنت...,Tech
45496,أعلنت شركة إيه أم دي، عن طرح أحدث منصات الكمبي...,Tech
45497,"أبوظبي ""الخليج"":زار الفريق سيف عبدالله الشعفار...",Tech
45498,تشارك وزارة الاقتصاد في الدورة ال 33 لأسبوع جي...,Tech


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45500 entries, 0 to 45499
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      45500 non-null  object
 1   category  45500 non-null  object
dtypes: object(2)
memory usage: 711.1+ KB


In [9]:
df.describe()

,text,category
count,45500,45500
unique,45485,7
top,إعداد: محمد صالح القرق دخل على رسول الله- صلى ...,Finance
freq,3,6500


In [10]:
df.isnull().sum()

text        0
category    0
dtype: int64

In [11]:
#lowercasing and removing special characters
df['text'] = df['text'].str.lower().str.replace(r'\W', ' ', regex=True)

In [12]:
df

,text,category
0,دبي الخليج حصدت شعاع كابيتال جائزة أفضل ش...,Finance
1,أبوظبي علي أسعد تراجعت أسواق المال في الدول...,Finance
2,استأنفت أسواق الأسهم المحلية أمس تحركها باتجاه...,Finance
3,دبي الخليج أعلنت شركة تكافل الإمارات عن بدء...,Finance
4,تحتفل شركة طاقة الخليج البحرية التي تتخذ من د...,Finance
...,...,...
45495,على هامش مشاركتها في أسبوع جيتكس للتقنية أعلنت...,Tech
45496,أعلنت شركة إيه أم دي عن طرح أحدث منصات الكمبي...,Tech
45497,أبوظبي الخليج زار الفريق سيف عبدالله الشعفار...,Tech
45498,تشارك وزارة الاقتصاد في الدورة ال 33 لأسبوع جي...,Tech


In [13]:
pip install transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 15.1 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import faiss
from transformers import DistilBertTokenizer, DistilBertModel
import torch


In [15]:
# Initialize DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [16]:
def encode_documents(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Take the mean of token embeddings as document embeddings
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
            embeddings.append(embedding)
    return embeddings


In [17]:
# Sample 1,000 rows
df = df.sample(n=1000, random_state=42) 

In [18]:
df.shape

(1000, 2)

In [19]:
# Encode documents
document_embeddings = encode_documents(df['text'].tolist())

In [20]:
# Create FAISS index
dimension = document_embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(document_embeddings))


In [21]:
# Save the index
faiss.write_index(index, 'document_index.faiss')

In [22]:
from transformers import pipeline

# Load FAISS index
index = faiss.read_index('document_index.faiss')

In [23]:
from transformers import pipeline

# Initialize text generation pipeline
generator = pipeline('text-generation', model='gpt2')

def generate_answer(query):
    # Encode the query
    query_embedding = encode_documents([query])[0]
    
    # Retrieve relevant documents
    _, indices = index.search(np.array([query_embedding]), k=5)  # Retrieve top 5 documents
    retrieved_docs = [df.iloc[idx]['text'] for idx in indices[0]]
    
    # Combine retrieved documents into a context for generation
    context = ' '.join(retrieved_docs)
    
    # Ensure the context is within the acceptable length
    if len(context) > 1024:
        context = context[:1024]  # truncate to fit model's max length

    # Generate response
    generated = generator(f"Context: {context}\n\nQuestion: {query}", max_length=200, max_new_tokens=150)
    
    return generated[0]['generated_text']

# Test the model
print(generate_answer("What is the main theme of the dataset?"))


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more informat

Context: ارتفع مؤشر الصقر الصادر عن المال كابيتال 43 45 نقطة توازي 2  ليغلق عند 2149 91 نقطة  وكانت أكثر الشركات ارتفاعا   الواحة بنسبة 8 8   الاتحاد العقارية 5 48   أملاك 4 69   دانة غاز 4 48    تصدر جهاز iomega ego portable hard drive 500gb لائحة أفضل خمسة أقراص صلبة خارجية قابلة للحمل والتنقل بحسب موقع بي سي ورلد 1iomega ego portable hard drive 500gb  السعة  500 غيغابايت    المنافذ المدعومة  firewire 400  firewire 800  usb 2  0  الملحقات  usb 2  0   power cable  firewire 400cable  firewire 800 cable   البرامج الملحقة  retrospect express  iomega quikprotect  الضمان  3 سنوات    السعر عند اعداد التقرير  150 دولار  2western digital mypassport essential se 1tb   السعة  1000غيغابايت    المنافذ المدعومة  usb 2  0  سرعة الدوران  5400 دورة في الدقيقة    الملحقات  mini usb cable  البرامج الملحقة  wd smartware   الضمان  سنة واحدة    السعر عند اعداد التقرير  25  178 دولار  3wiebetech toughtechxe mini 500gb   السعة  500 غيغابايت    المنافذ المدعومة  external sata 300  firewire 800  usb 2  0  سرع

In [24]:
# Test the model
print(generate_answer("التهاب الكبد الوبائي"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Context: صادقت الجمعية العمومية لشركة  دبي الوطني للتأمين وإعادة التأمين  المنعقدة أمس على توزيع أرباح نقدية للمساهمين بنسبة 20  من رأس المال عن عام 2013   أعلن سوق دبي المالي عن إيداع أسهم المنحة لشركة بيت التمويل الخليجي بنسبة 20  في حسابات المساهمين وذلك كما في تاريخ إغلاق سجل الشركة في 22 2 2009   أعلن مجلس إدارة شركة سوق دبي المالي أنه اجتمع يوم الاثنين بتاريخ 27 9 2010 واعتمد البنود المتعلقة بتوصيات لجنة الترشيحات والمكافآت  والهيكل التنظيمي للشركة   ينظم بيت الشعر ندوة حوارية بعنوان دغدغة قصيدة النثر  يشترك فيها د   بهجت الحديثي وعبداللطيف الزبيدي ويقدم الندوة ويديرها عبدالفتاح صبري  وذلك في الثامنة من مساء اليوم   ينعقد اجتماع شركة صروح العقارية يوم 27 يوليو تموز الجاري  وذلك لمناقشة الانجازات المالية للربع الثاني من عام  2011 ومراجعة أمور عامة وتشغيلية واستراتيجية  

Question: التهاب الكبد الوبائي�اري - ها وا الينواء فيأوول السقاصه الناتد الاةن، الجمعية العمومية لصادقت الجمعية لشركة بيت التمويل الخ�اري

5. يولقبت النصاده اجمع الؽوعة قنام نر عطيرة بحلام �


In [3]:
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Path to the main dataset folder
dataset_path = '/kaggle/input/sanad-dataset'  # Update this path to the location of your dataset

# Categories (folder names)
categories = ['Culture', 'Finance', 'Medical', 'Politics', 'Religion', 'Sports', 'Tech']

# Initialize a list to hold the data in chunks
data_chunks = []

# Process each category folder
for category in categories:
    category_path = os.path.join(dataset_path, category)
    for file_name in os.listdir(category_path):
        file_path = os.path.join(category_path, file_name)
        # Ensure it's a text file
        if file_name.endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                # Append the content and category to the data list
                data_chunks.append({'text': content, 'category': category})
        
        # Save in chunks to avoid memory issues
        if len(data_chunks) >= 1000:  # Adjust the chunk size as needed
            chunk_df = pd.DataFrame(data_chunks)
            chunk_df.to_csv('sanad_combined2.csv', mode='a', index=False, header=not os.path.exists('sanad_combined.csv'))
            data_chunks = []  # Clear the list to free up memory

# Save any remaining data
if data_chunks:
    chunk_df = pd.DataFrame(data_chunks)
    chunk_df.to_csv('sanad_combined2.csv', mode='a', index=False, header=not os.path.exists('sanad_combined.csv'))

print("Data successfully combined and saved in chunks!")


KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Load the dataset and skip the first row
data = pd.read_csv('sanad_combined2.csv', delimiter=',', skiprows=1, names=['text', 'category'])

# Verify that the data is loaded correctly
print(data.head())


In [ ]:
data

In [1]:
import torch 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [2]:
import os
import pandas as pd
import numpy as np
# Function to load data from folders
def load_data_from_folders(main_folder_path):
    data = []
    for category_folder in os.listdir(main_folder_path):
        category_label = category_folder  # Assuming folder name is the category label
        category_path = os.path.join(main_folder_path, category_folder)
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                data.append({'text': content, 'category': category_label})
    return pd.DataFrame(data)

# Main folder containing subfolders for each category
main_folder_path = '/kaggle/input/sanad-dataset'

# Load data from folders
data = load_data_from_folders(main_folder_path)

In [3]:
data

,text,category
0,"دبي - ""الخليج"":حصدت شعاع كابيتال جائزة ""أفضل ش...",Finance
1,أبوظبي - علي أسعد: تراجعت أسواق المال في الدول...,Finance
2,استأنفت أسواق الأسهم المحلية أمس تحركها باتجاه...,Finance
3,دبي «الخليج»: أعلنت شركة تكافل الإمارات عن بدء...,Finance
4,تحتفل شركة طاقة الخليج البحرية، التي تتخذ من د...,Finance
...,...,...
45495,على هامش مشاركتها في أسبوع جيتكس للتقنية أعلنت...,Tech
45496,أعلنت شركة إيه أم دي، عن طرح أحدث منصات الكمبي...,Tech
45497,"أبوظبي ""الخليج"":زار الفريق سيف عبدالله الشعفار...",Tech
45498,تشارك وزارة الاقتصاد في الدورة ال 33 لأسبوع جي...,Tech


In [4]:
pip install faiss-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 17.7 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install --upgrade ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 1.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.8
    Uninstalling widgetsnbextension-3.6.8:
      Successfully uninstalled widgetsnbextension-3.6.8
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.11
    Uninstalling jupyterlab_widgets-3.0.11:
      Successfully uninstalled jupyterlab_widgets-3.0.11
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency

In [6]:
import pandas as pd
import faiss
from transformers import DistilBertTokenizer, DistilBertModel, pipeline
import torch
import numpy as np
import gc

# Initialize DistilBERT
print("Initializing DistilBERT model and tokenizer...")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)
print("Model and tokenizer initialized.")


def encode_batch(texts):
    print(f"Encoding batch of size: {len(texts)}")
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Move back to CPU before converting to numpy
    print("Batch encoding complete.")
    return embeddings

def process_and_index_chunk(start_index, end_index, index):
    print(f"Processing chunk from index {start_index} to {end_index}...")
    batch_texts = data['text'][start_index:end_index].tolist()
    
    # Split batch into smaller sub-batches to avoid OOM
    sub_batch_size = 100  # You can adjust this
    for i in range(0, len(batch_texts), sub_batch_size):
        sub_batch_texts = batch_texts[i:i + sub_batch_size]
        batch_embeddings = encode_batch(sub_batch_texts)
        print(f"Adding {len(batch_embeddings)} embeddings to the FAISS index...")
        index.add(batch_embeddings)
        
        # Free up memory
        del batch_embeddings
        gc.collect()  # Call garbage collector
        print(f"Sub-batch {i // sub_batch_size + 1} processed and indexed.")

def generate_answer(query, index, data):
    print(f"Generating answer for query: {query}")
    
    # Encode the query
    query_embedding = encode_batch([query])[0]
    
    # Retrieve relevant documents (limit to top 3 for faster generation)
    print("Searching for relevant documents...")
    _, indices = index.search(np.array([query_embedding]), k=3)
    retrieved_docs = [data.iloc[idx]['text'] for idx in indices[0]]
    print(f"Retrieved documents: {retrieved_docs}")
    
    # Truncate the context if it's too long
    context = ' '.join(retrieved_docs)
    if len(context) > 1024:
        context = context[:1024]
    
    # Generate response
    print("Generating response using GPT-2...")
    generator = pipeline('text-generation', model='gpt2', device=device)  # Use GPU if available
    generated = generator(f"Context: {context}\n\nQuestion: {query}", max_new_tokens=150)
    
    print("Response generated.")
    return generated[0]['generated_text']


Initializing DistilBERT model and tokenizer...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Model and tokenizer initialized.


In [7]:
# Initialize the FAISS index
dimension = 768  # DistilBERT's output dimension
index = faiss.IndexFlatL2(dimension)
print("FAISS index initialized.")

# Process data in chunks
chunk_size = 1000  # Adjust based on your available memory
for start in range(0, len(data), chunk_size):
    end = min(start + chunk_size, len(data))
    process_and_index_chunk(start, end, index)
    print(f"Processed chunk from {start} to {end}.")

# Save the index
print("Saving the FAISS index...")
faiss.write_index(index, 'document_index_optimized.faiss')
print("FAISS index saved.")



FAISS index initialized.
Processing chunk from index 0 to 1000...
Encoding batch of size: 100
Batch encoding complete.
Adding 100 embeddings to the FAISS index...
Sub-batch 1 processed and indexed.
Encoding batch of size: 100
Batch encoding complete.
Adding 100 embeddings to the FAISS index...
Sub-batch 2 processed and indexed.
Encoding batch of size: 100
Batch encoding complete.
Adding 100 embeddings to the FAISS index...
Sub-batch 3 processed and indexed.
Encoding batch of size: 100
Batch encoding complete.
Adding 100 embeddings to the FAISS index...
Sub-batch 4 processed and indexed.
Encoding batch of size: 100
Batch encoding complete.
Adding 100 embeddings to the FAISS index...
Sub-batch 5 processed and indexed.
Encoding batch of size: 100
Batch encoding complete.
Adding 100 embeddings to the FAISS index...
Sub-batch 6 processed and indexed.
Encoding batch of size: 100
Batch encoding complete.
Adding 100 embeddings to the FAISS index...
Sub-batch 7 processed and indexed.
Encoding b

In [8]:
# Test the optimized model
query = "ماذا حدث في اغتيال البرلماني عبد اللطيف مرداس"
print("Testing the model...")
answer = generate_answer(query, index, data)
print(f"Answer: {answer}")

Testing the model...
Generating answer for query: ماذا حدث في اغتيال البرلماني عبد اللطيف مرداس
Encoding batch of size: 1
Batch encoding complete.
Searching for relevant documents...
Retrieved documents: ['تنظم ندوة الثقافة والعلوم في مقرها في دبي عند العاشرة والنصف من صباح اليوم مؤتمراً صحفياً للإعلان عن الفائزين في جائزة العويس للإبداع .', 'ينظم النادي الثقافي العربي أمسية للشاعر حسن عبدالسلام أبودية عند الثامنة من مساء يوم غد في مقر النادي في الشارقة .', 'استعرض ابن علي السيارة النموذجية «رين سبيد إيتوس» ذات الابتكارات الفائقة والتقنيات الجديدة التي تغير عالم التنقل.']
Generating response using GPT-2...


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response generated.
Answer: Context: تنظم ندوة الثقافة والعلوم في مقرها في دبي عند العاشرة والنصف من صباح اليوم مؤتمراً صحفياً للإعلان عن الفائزين في جائزة العويس للإبداع . ينظم النادي الثقافي العربي أمسية للشاعر حسن عبدالسلام أبودية عند الثامنة من مساء يوم غد في مقر النادي في الشارقة . استعرض ابن علي السيارة النموذجية «رين سبيد إيتوس» ذات الابتكارات الفائقة والتقنيات الجديدة التي تغير عالم التنقل.

Question: ماذا حدث في اغتيال البرلماني عبد اللطيف مرداس وأتفاثي بن أوضكاب الأوتسانا ين وتهيف الأخما بن أبودية أبودية أبيته والعربي انما جسداخ العقوم بحدث اللم.

[5] In other words, صالم المسفلوافن was given to أبودية, and not to اءكتف.


In [9]:
from transformers import pipeline

# Load FAISS index
index = faiss.read_index('document_index_optimized.faiss')

def generate_answer(query):
    # Encode the query
    query_embedding = encode_batch([query])[0]
    
    # Retrieve relevant documents (limit to top 3 for faster generation)
    _, indices = index.search(np.array([query_embedding]), k=3)
    retrieved_docs = [data.iloc[idx]['text'] for idx in indices[0]]
    
    # Combine the retrieved documents into a context
    context = ' '.join(retrieved_docs)
    
    # Truncate the context if it's too long (important for Arabic)
    if len(context) > 1024:
        context = context[:1024]
    
    # Use an Arabic-specific model for generation
    generator = pipeline('text-generation', model='aubmindlab/aragpt2-base', device=0)
    prompt = f"السياق: {context}\n\nالسؤال: {query}\n\nالجواب:"
    generated = generator(prompt, max_new_tokens=150, pad_token_id=50256)
    
    return generated[0]['generated_text']

# Test the model with an Arabic question
print(generate_answer("ما هو مشروع الحملة الوطنية للتخلص من الالتهاب الكبدي الوبائي «سي»؟"))


Encoding batch of size: 1
Batch encoding complete.


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

السياق: كان سهم بنك الخليج الأول أمس نجم التداولات في الأسواق المحلية إذ بلغت 4 .940 مليون درهم نتيجة صفقة تحويل 56 مليون سهم، إلا أن السهم تراجع بنسبة 3 .0% إلى 7 .15 درهم .. .وكابيتال انتليجنس ترفع تصنيفهرفعت وكالة كابيتال انتليجنس تصنيف القوة المالية لبنك الخليج الأول من درجة A إلى درجة A+، ومنحته نظرة مستقبلية مستقرة، في ضوء التحسن العام للأسس المالية للبنك، وخصوصاً فيما يتعلق بجودة الأصول والربحية والسيولة المالية . دبي - "الخليج":أجل اجتماع الجمعية العمومية لشركة "شعاع كابيتال" الذي كان مقرراً أمس الأول إلى الاثنين 7 إبريل/ نيسان المقبل . أبوظبي - "الخليج":أقرت الجمعية العمومية لشركة دار التمويل اقتراح مجلس الإدارة بتوزيع أرباح نقدية بنسبة 20% وأسهم منحة بنسبة 5 .2% عن العام الماضي .

السؤال: ما هو مشروع الحملة الوطنية للتخلص من الالتهاب الكبدي الوبائي «سي»؟

الجواب: هل أنت مصاب بالأنيميا ؟ وما هي أعراض مرضك ؟ وهل لديك نقص في فيتامين ب 12 ؟ كل ما عليك القيام به هو استشارة الطبيبالأمراض التي تصيب الكبد والكلى والجهاز الهضمي والقلب والسرطان والسكري وأمراض الجهاز التنفسيإذا كنت مصاب

In [10]:

# Test the model with an Arabic question
print(generate_answer("ما هي مواعيد معهد الشارقة للفنون؟"))


Encoding batch of size: 1
Batch encoding complete.
السياق: تنظم ندوة الثقافة والعلوم في مقرها في دبي عند العاشرة والنصف من صباح اليوم مؤتمراً صحفياً للإعلان عن الفائزين في جائزة العويس للإبداع . أقرت الجمعية العمومية للمؤسسة الوطنية للسياحة والفنادق توزيع 60% أرباحاً نقدية على مساهميها يوم أمس الأول . تقرر عقد اجتماع مجلس إدارة دار التأمين الخميس المقبل الساعة الثانية والنصف بعد الظهر لمناقشة الموازنة التقديرية لسنة 2013 .

السؤال: ما هي مواعيد معهد الشارقة للفنون؟

الجواب: عقدت اللجنة التنفيذية لمعهد الشارقة للفنون اجتماعها الدوري الخامس عشر برئاسة د. عبد الله بن محمد العويس رئيس مجلس إدارة المعهد ، حيث ناقش الاجتماع عددا من الموضوعات المدرجة على جدول أعماله واتخذ بشأنها القرارات والتوصيات المناسبة.


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "aubmindlab/aragpt2-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss
    return torch.exp(loss).item()

text = "مجلوبة من موقع [رابط]! ؟ ( 1 2 3 4 5... الصفحة الأخيرة )عنوان الموضوع المشاركة الأخيرة تاريخ بداية الموضوع عدد الردود عدد المشاهدات كاتب الموضوع تقييم الموضوعآخر يوم آخر يومين آخر أسبوع آخر 10 أيام آخر أسبوعين آخر شهر آخر 45 يوم آخر شهرين آخر 75 يوم آخر 100 يوم السنة الماضية البدايةلوحة تحكم العضو الرسائل الخاصة الاشتراكات المتواجدون الآن البحث في المنتدى الصفحة الرئيسية للمنتدى منتدى الأسهم منتدى الأخبار الاقتصادية منتدى ألاكاديمية للدروس والتحليل الفني تسجيلات والبث المباشر للقوائم اثناء التداول منتدى الأسهم السعودية أساتذة الأكاديمية.. في القنوات الإعلامية. منتدى برامج الأسهم وشروحاتها برنامج الميتاستوك برنامج ويلث لاب ( Wealth - Lab ) برنامج الإيمي بروكر برنامج MotiveWave برنامج تكرتشارت برنامج"
perplexity = calculate_perplexity(text)
print(f"Perplexity: {perplexity}")


Perplexity: 5.820170879364014


In [12]:
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu_score(references, hypotheses):
    references = [[ref.split()] for ref in references]
    hypotheses = [hyp.split() for hyp in hypotheses]
    score = corpus_bleu(references, hypotheses)
    return score

# Example
references = ["ما هو مشروع الحملة الوطنية للتخلص من الالتهاب الكبدي الوبائي «سي»؟"]
hypotheses = ["مجلوبة من موقع [رابط]! ؟ ( 1 2 3 4 5... الصفحة الأخيرة )عنوان الموضوع المشاركة الأخيرة تاريخ بداية الموضوع عدد الردود عدد المشاهدات كاتب الموضوع تقييم الموضوعآخر يوم آخر يومين آخر أسبوع آخر 10 أيام آخر أسبوعين آخر شهر آخر 45 يوم آخر شهرين آخر 75 يوم آخر 100 يوم السنة الماضية البدايةلوحة تحكم العضو الرسائل الخاصة الاشتراكات المتواجدون الآن البحث في المنتدى الصفحة الرئيسية للمنتدى منتدى الأسهم منتدى الأخبار الاقتصادية منتدى ألاكاديمية للدروس والتحليل الفني تسجيلات والبث المباشر للقوائم اثناء التداول منتدى الأسهم السعودية أساتذة الأكاديمية.. في القنوات الإعلامية. منتدى برامج الأسهم وشروحاتها برنامج الميتاستوك برنامج ويلث لاب ( Wealth - Lab ) برنامج الإيمي بروكر برنامج MotiveWave برنامج تكرتشارت برنامج"]
bleu_score = calculate_bleu_score(references, hypotheses)
print(f"BLEU Score: {bleu_score}")


BLEU Score: 0.3067117324758864


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
